In [1]:
import torch

print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))
print(torch.version.cuda)

2.10.0+cu128
True
0
NVIDIA GeForce RTX 4070 Laptop GPU
12.8


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [4]:
from train_model import CustomCNN

In [5]:
# Load the checkpoint
checkpoint = torch.load('/home/muhammad_adib/imgclass_cnn/intel_cnn_model.pth', map_location=device)

# Re-initialize the model architecture (must match the saved model)
model = CustomCNN(num_classes=len(checkpoint['class_names'])).to(device)

# Load the saved weights
model.load_state_dict(checkpoint['model_state_dict'])

# Set to evaluation mode
model.eval()

# Access saved information
print(f"Classes: {checkpoint['class_names']}")
print(f"Test accuracy: {checkpoint['test_accuracy']*100:.2f}%")

Classes: ['buildings', 'forest', 'mountain', 'sea', 'street']
Test accuracy: 95.12%
